Source code :https://github.com/Adapter-Hub/adapters/tree/main/notebooks

In [ ]:
!pip install -Uq adapters
!pip install -q datasets
!pip install -q accelerate
!pip install --upgrade pip setuptools wheel
!pip install pyarrow

In [ ]:
import pandas as pd
from datasets import Dataset
from adapters import AutoAdapterModel
from transformers import AutoTokenizer, TrainingArguments, EvalPrediction
from adapters import AdapterTrainer
import torch
import numpy as np
from adapters.composition import Fuse
from sklearn.model_selection import train_test_split

In [ ]:
trainDf = pd.read_excel('/content/train_set(2).xlsx')

In [ ]:
trainDf.head()

,Label,Text,English_Translation,Telugu_Translation
0,NTL,We need Mr chari s review on master,We need Mr Charis review on Master,మాస్టర్పై చారి గారి సమీక్ష కావాలి
1,NEG,worst government YSRCP chala chethha ga parip...,Worst government YSRCP is managing very badly,వర్స్ట్ ప్రభుత్వం YSRCP చాలా చెత్తగా పరిపాలన ...
2,NEG,bayya nuvvu emina cheppu kani bagoledu ani che...,You tell me what to say but its difficult to s...,బయ్య నువ్వు ఏమినా చెప్పు కనీ బగోలెడు అని చెప్ప...
3,POS,Dube gadini vadilesi manchhi Pani chesaru RCBT...,The Dube river is overflowing and RCB has done...,డ్యూబే గడిని వడిలేసి మంచి పని చేశారు ఆర్సీబీ ట...
4,POS,I came to watch thyview s review crying after ...,I came to watch Thyviews review crying after w...,నేను దిల్ బేచారా చూసిన తర్వాత థైవ్యూ సమీక్ష చూ...


In [ ]:
testDf = pd.read_excel('/content/test_set(1).xlsx')

In [ ]:
# Here, I did mapping of sentiment labels to integers.
labelMapping = {"POS": 1, "NEG": 0,"NTL":2}

In [ ]:
# Here, I applied the mapping to the train and test datasets
trainDf["Label"] = trainDf["Label"].map(labelMapping)
testDf["Label"] = testDf["Label"].map(labelMapping)

In [ ]:
#Here, I split the training data into training and validation sets into 80 & 20 percent
trainDf, valDf = train_test_split(trainDf, test_size=0.2, random_state=42)

In [ ]:
trainDf.head()

,Label,Text,English_Translation,Telugu_Translation
696,1,Bro naakoka heart ivvu tvaragaa,Bro my heart is hurting a lot,బ్రో నాకోక హార్ట్ ఇవ్వు త్వరగా
15497,0,Ravicherry Varun Sandesh tho competition anta ...,I have to prove myself to MD that Im not a lan...,రవిచెర్రి వరుణ్ సందేష్ తో కోటిపాటి పోటీ ఉంది మ...
8409,2,in krack movie how many of you have seen in c...,In the Krack movie how many of you have seen i...,క్రాక్ సినిమాలో క్లైమాక్స్ లో ఎంతమందికి కనిపిం...
13448,1,Jagadish Credits nak icchav enti anna,What are the credits that Jagadish wants brother,జగదీష్ క్రెడిట్స్ నక్ ఇచ్చవ్ అంటే అన్నా
15455,0,gedhala JanaSenaParty PawanKalyan Telusu Ambu...,Pawan Kalyan of JanaSenaParty said in Telugu t...,జనసేన పార్టీ అధినేత పవన్ కళ్యాణ్ తెలుసు ఆంబులె...


In [ ]:
# Here, I converted the dataframes to datasets
trainDataset = Dataset.from_pandas(trainDf)
valDataset = Dataset.from_pandas(valDf)
testDataset = Dataset.from_pandas(testDf)

In [ ]:
# Load tokenizer and model for xlmr-base with adapters
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoAdapterModel.from_pretrained("xlm-roberta-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.load_adapter("Blue7Bird/my-Telugu-codemix-xlmr-adapter2", load_as="myCodemixAdapter", set_active=True)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/923 [00:00<?, ?B/s]

head_config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

'myCodemixAdapter'

In [ ]:
numLabels = 3
model.add_classification_head(
    "Telugu_codemix_SentimentTask",
    num_labels=numLabels,
    id2label={"POS": 1, "NEG": 0, "NTL": 2}
)

In [ ]:
# Here I freeze the pre-trained model weights
model.train_adapter("myCodemixAdapter")

In [ ]:
# Here, I built a function to encode datasets
def encodeBatch(batch):
    encoding = tokenizer(
        batch["Text"],
        max_length=180,
        truncation=True,
        padding="max_length",
    )
    encoding["labels"] = batch["Label"]
    return encoding


In [ ]:
# Here, I encoded datasets
trainDataset = trainDataset.map(encodeBatch, batched=True)
valDataset = valDataset.map(encodeBatch, batched=True)
testDataset = testDataset.map(encodeBatch, batched=True)

Map:   0%|          | 0/13494 [00:00<?, ? examples/s]

Map:   0%|          | 0/3374 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# Here, I Set the format for PyTorch
trainDataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
valDataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
testDataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
# Here, I Define metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc, 'F1': f1, 'Precision': precision, 'Recall': recall}


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments, EarlyStoppingCallback, TrainerCallback

In [ ]:
# Here, I Set up training arguments
trainingArgs = TrainingArguments(
    learning_rate=0.0001,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    output_dir="./training_output",
    logging_dir='./logs',
    overwrite_output_dir=True,
    remove_unused_columns=False,
    fp16=True,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    eval_steps=10,
    load_best_model_at_end=True,  # Here I load the best model
    warmup_steps=500,
    weight_decay=0.001,
)

# Here, I initialized the trainer with the training arguments
trainer = AdapterTrainer(
    model=model,
    args=trainingArgs,
    train_dataset=trainDataset,
    eval_dataset=valDataset,  # Here I used the validation dataset for evaluation during training
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=100)]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
10,2.213700,1.095445,0.401897,0.401897,0.401897,0.401897
20,2.216700,1.072156,0.427682,0.427682,0.427682,0.427682
30,2.151200,1.036401,0.458210,0.458210,0.458210,0.458210
40,2.073600,0.994233,0.554831,0.554831,0.554831,0.554831
50,2.011600,0.950485,0.625963,0.625963,0.625963,0.625963
60,1.951900,0.907679,0.635151,0.635151,0.635151,0.635151
70,1.871000,0.864832,0.644932,0.644932,0.644932,0.644932
80,1.886800,0.819812,0.673385,0.673385,0.673385,0.673385
90,1.760800,0.776252,0.679905,0.679905,0.679905,0.679905
100,1.688700,0.732152,0.697392,0.697392,0.697392,0.697392


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=4960, training_loss=0.9460548767159062, metrics={'train_runtime': 3485.1693, 'train_samples_per_second': 77.437, 'train_steps_per_second': 2.422, 'total_flos': 1.51717716395424e+16, 'train_loss': 0.9460548767159062, 'epoch': 11.753554502369669})

In [ ]:
# Here, I evaluated the model on validation set
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.47041428089141846, 'eval_Accuracy': 0.8100177830468287, 'eval_F1': 0.8100177830468287, 'eval_Precision': 0.8100177830468287, 'eval_Recall': 0.8100177830468287, 'eval_runtime': 5.817, 'eval_samples_per_second': 580.02, 'eval_steps_per_second': 36.273, 'epoch': 11.753554502369669}


In [ ]:
# Here, I evaluated the model on the test dataset
testEvalResult = trainer.evaluate(eval_dataset=testDataset)
print(testEvalResult)

{'eval_loss': 0.5029357671737671, 'eval_Accuracy': 0.7973333333333333, 'eval_F1': 0.7973333333333333, 'eval_Precision': 0.7973333333333333, 'eval_Recall': 0.7973333333333333, 'eval_runtime': 5.3808, 'eval_samples_per_second': 557.542, 'eval_steps_per_second': 34.939, 'epoch': 11.753554502369669}
